this outputs main_trial_level data (main task) of both YAs and OAs

In [1]:
import pandas as pd
import numpy as np
from os import path,listdir

making pathways/ addresses

In [2]:
dataset_list = [
    path.join('..','..','econdec','sourcedata','ds1'),
    path.join('..','sourcedata'),
]
derivs_dir = path.join('..','derivatives')

In [3]:
source_dir = dataset_list[0]

In [4]:
dataframes_list = []
for dataset in dataset_list:
    source_dir = dataset
    sub_list = [s for s in listdir(source_dir) if s.startswith('sub-')]
    for sub in sub_list:
        sub_dir = path.join(source_dir,sub)
        file_list = [f for f in listdir(sub_dir) if f.endswith('_task-main_beh.xlsx')]
        for file in file_list:
            file_path = path.join(sub_dir,file)
            print(file_path)
            dataframes_list.append(pd.read_excel(file_path))
print('DONE!')

..\..\econdec\sourcedata\ds1\sub-100\sub-100_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-101\sub-101_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-102\sub-102_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-103\sub-103_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-104\sub-104_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-105\sub-105_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-106\sub-106_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-107\sub-107_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-108\sub-108_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-109\sub-109_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-110\sub-110_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-111\sub-111_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-112\sub-112_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-113\sub-113_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-114\sub-114_task-main_beh.xlsx
..\..\econdec\sourcedata\ds1\sub-115\sub

..\sourcedata\sub-455\sub-455_task-main_beh.xlsx
..\sourcedata\sub-456\sub-456_task-main_beh.xlsx
..\sourcedata\sub-457\sub-457_task-main_beh.xlsx
..\sourcedata\sub-458\sub-458_task-main_beh.xlsx
..\sourcedata\sub-459\sub-459_task-main_beh.xlsx
..\sourcedata\sub-460\sub-460_task-main_beh.xlsx
..\sourcedata\sub-461\sub-461_task-main_beh.xlsx
..\sourcedata\sub-462\sub-462_task-main_beh.xlsx
..\sourcedata\sub-463\sub-463_task-main_beh.xlsx
..\sourcedata\sub-464\sub-464_task-main_beh.xlsx
..\sourcedata\sub-465\sub-465_task-main_beh.xlsx
..\sourcedata\sub-467\sub-467_task-main_beh.xlsx
..\sourcedata\sub-469\sub-469_task-main_beh.xlsx
..\sourcedata\sub-470\sub-470_task-main_beh.xlsx
..\sourcedata\sub-471\sub-471_task-main_beh.xlsx
..\sourcedata\sub-472\sub-472_task-main_beh.xlsx
..\sourcedata\sub-473\sub-473_task-main_beh.xlsx
..\sourcedata\sub-475\sub-475_task-main_beh.xlsx
..\sourcedata\sub-476\sub-476_task-main_beh.xlsx
..\sourcedata\sub-477\sub-477_task-main_beh.xlsx
..\sourcedata\sub-47

In [5]:
concat_df = pd.concat(dataframes_list)

In [6]:
def clean_dates(date):
    if isinstance(date, str):
        return(date)
    else: 
        return('_'.join(str(date).split(' ')[0].split('-')[1:]))

In [7]:
concat_df.Date = np.vectorize(clean_dates)(concat_df['Date'])
concat_df.StockPic = np.vectorize(path.basename)(concat_df['StockPic'])
concat_df.BondPic = np.vectorize(path.basename)(concat_df['BondPic'])
concat_df.FractalChosen = np.vectorize(path.basename)(concat_df['FractalChosen'])

## Fix the data type issue:
we convert ProbGood to all floats, and divide by 100, and then return null where the number is out of range:

In [8]:
concat_df['ProbGood'] = (pd.to_numeric(concat_df['ProbGood'], errors='coerce')) / 100.0
concat_df['ProbGood'] = concat_df['ProbGood'].where((concat_df['ProbGood'] >= 0.0) & (concat_df['ProbGood'] <= 1.0))

In [9]:
concat_df ['val_estdiff_valid'] = concat_df.ProbGood - concat_df.TrueProbGood

In [10]:
concat_df['absolute_error'] = concat_df['val_estdiff_valid'].abs() #here I make absolute_error to be the absolute value of val_estdiff_valid
concat_df.head()

,SubjNum,AgeGroup,ExperimenterName,RunNum,Date,Time,TrialNum,TrialNumbydomdist,Domain,Magnitude,...,ConfidenceST,ConfidenceRT,StockNumber,BondNumber,GenderJudgment,TotalPayout,TrueProbGood,EstWithinRange?,val_estdiff_valid,absolute_error
0,100,1,kf,1,10_12,11:31:01.963000,1,1,LOSS,low,...,2.141471e+06,3.022637,16,9,1,-6,0.300000,0,2.000001e-01,2.000001e-01
1,100,1,kf,1,10_12,11:31:01.963000,2,2,LOSS,low,...,2.141525e+06,3.695852,16,9,1,-12,0.155172,0,1.448277e-01,1.448277e-01
2,100,1,kf,1,10_12,11:31:01.963000,3,3,LOSS,low,...,2.141546e+06,3.121775,16,9,1,-18,0.300000,1,1.038193e-07,1.038193e-07
3,100,1,kf,1,10_12,11:31:01.963000,4,4,LOSS,low,...,2.141574e+06,3.406241,16,9,1,-24,0.500000,0,-1.000000e-01,1.000000e-01
4,100,1,kf,1,10_12,11:31:01.963000,5,5,LOSS,low,...,2.141602e+06,4.553061,16,9,1,-26,0.700000,0,-1.000001e-01,1.000001e-01


In [11]:
output_path = path.join(derivs_dir,'trialwise','main_trial_level.csv')
concat_df.to_csv(output_path, index=0)